In [ ]:
from faker import Faker
import mysql.connector
import re
import unicodedata
from datetime import datetime

# Połączenie z bazą danych
con = mysql.connector.connect(
    host="giniewicz.it",
    user="team19",
    password="te@mzaiq",
    database="team19"
)

mycursor = con.cursor()

# Inicjalizacja obiektu Faker
fake = Faker('pl_PL')  # Polski lokalizator

# Liczba rekordów do wygenerowania
num_records = 100

# Funkcja do usuwania polskich znaków diakrytycznych z tekstu
def remove_polish_chars(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'
    )

# Usuwanie wszystkich danych z tabeli Klienci przed dodaniem nowych
mycursor.execute("TRUNCATE TABLE Klienci")

# Generowanie i wstawianie nowych danych do tabeli
for _ in range(num_records):
    Imie = fake.first_name()
    Nazwisko = fake.last_name()
    
    # Generowanie e-maila na podstawie imienia i nazwiska, usuwanie polskich znaków
    Email = f"{remove_polish_chars(Imie.lower())}.{remove_polish_chars(Nazwisko.lower())}@domena.pl"
    
    # Generowanie numeru telefonu z prefiksem +48
    Numer_telefonu = fake.phone_number()
    Numer_telefonu = re.sub(r'\D', '', Numer_telefonu)  # Usuwamy wszystko oprócz cyfr
    Numer_telefonu = "+48" + Numer_telefonu[-9:]  # Dodajemy prefiks +48 i ograniczamy do 9 cyfr

    Adres = fake.address().replace("\n", ", ")
    
    # Osoba bliska ma to samo nazwisko i numer telefonu zaczynający się od +48
    Dane_kontaktowe_bliskich = f"{remove_polish_chars(fake.first_name())} {Nazwisko}, +48{re.sub(r'\D', '', fake.phone_number())[-9:]}"
    
    Data_urodzenia = fake.date_of_birth(minimum_age=18, maximum_age=80).strftime('%Y-%m-%d')

    # Wstawianie danych do tabeli Klienci
    mycursor.execute("""
        INSERT INTO Klienci (Imie, Nazwisko, Numer_telefonu, Email, Adres, Dane_kontaktowe_bliskich, Data_urodzenia)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (Imie, Nazwisko, Numer_telefonu, Email, Adres, Dane_kontaktowe_bliskich, Data_urodzenia))

# Zatwierdź zmiany i zamknij połączenie
con.commit()
print(f"Wstawiono {num_records} rekordów do tabeli Klienci.")

mycursor.close()
con.close()
